BD_OPERACOES em linhas

In [7]:
import sidrapy
import os

In [17]:
os.path.exists(f'{SALVA_OS_DFS}\\{nome}.xlsx')


False

In [15]:
nome = 'w'
f'{SALVA_OS_DFS}\\{nome}.xlsx'

'c:\\Users\\lumm\\OneDrive - Elera\\Documentos\\Python Scripts\\book_trading_python\\Dataframe\\w.xlsx'

In [10]:
#Abre os dataframes
bd_operacoes = pd.read_pickle('bd_operacoes.pickle')
curva_forward = pd.read_pickle('curva_forward.pickle')

In [29]:
import pandas as pd
import holidays
from workalendar.america import Brazil
from datetime import date, datetime
import os
import sys

from ehubAPI import forwardCurve

#Logins e Senhas
EHUB_EMAIL = "guilherme.cruz@elera.com"
EHUB_SENHA = "Ber@2021!"

#Diretórios
FORECAST_ASSUMPTIONS_DIR = r"C:\Users\lumm\OneDrive - Elera\Documentos\Python Scripts\Book no BI\Recursos\Forecast Assumptions.xlsx"
BD_OPERACOES_DIR = r"C:\Users\lumm\OneDrive - Elera\Documentos\Python Scripts\Book no BI\Recursos\BD_OPERACOES.xlsm"
INFLACOES_DFS_DIR = r'Z:\Risk\28. Portfolio Data Base\09.Inflação\Dataframe'
SALVA_OS_DFS = f'{os.getcwd()}\Dataframe'

#Constantes
MES_INICIAL = '2022-01-01'

ANOS_BD_OPERACOES = 5
MESES_BD_OPERACOES = ANOS_BD_OPERACOES * 12

ANOS_OPERACIONAL_BOOK = 2
MESES_OPERACIONAL_BOOK = ANOS_OPERACIONAL_BOOK * 12

ANOS_TOTAL_BOOK = 5
MESES_TOTAL_BOOK = ANOS_TOTAL_BOOK * 12

COL_INICIO_VOLUME = 26
COL_INICIO_PRECO = 92

In [30]:
def _salva_dataframes(df, nome, caminho=None, formato='Excel'):
    
    try:
        if not os.path.exists(caminho):
            os.makedirs(caminho)

        if formato == 'Excel':
            df.to_excel(f"{caminho}/{nome}.xlsx")

        elif formato == 'csv':
            df.to_csv(f"{caminho}/{nome}.csv")

        else:
            df.to_pickle(f"{caminho}/{nome}.df")
        print(f'> O Dataframe "{nome}" foi salvo na pasta {caminho}.\n')
    
    except:
        print('> Não foi possivel salvar. Algum parametro esta errado.')

In [17]:
# Monta um dataframe com todas os meses e horas dos meses 
def _periodo_e_horas_meses(mes_inicial=MES_INICIAL, qtd_meses=MESES_TOTAL_BOOK):
       
    try:
        col_periodo = []
        col_horas_mes = []
        mes = mes_inicial

        if isinstance(mes_inicial, str):
           mes = datetime.strptime(mes, '%Y-%m-%d')

        for i in range(qtd_meses):

            horas = ((pd.to_datetime(mes) + pd.DateOffset(months=1)) - mes)*24
            horas_int = int(str(horas)[:3])

            col_horas_mes.append(horas_int)
            col_periodo.append(mes)
            
            mes = pd.to_datetime(mes) + pd.DateOffset(months=1)

        periodos_e_horas = pd.DataFrame({'Periodo': col_periodo, 'Horas': col_horas_mes})
        print('> Dataframe de meses e horas montado com sucesso')
        
        #Salva o Dataframe
        _salva_dataframes(df=periodos_e_horas, nome="periodos_e_horas",formato='Excel')

        # Retorna um Dataframe
        return periodos_e_horas

    except:
        print('> Data precisa estar formatada em AAAA-MM-DD e quantidade de meses precisa ser um numero inteiro positivo')
        sys.exit()


In [36]:
# Monta a BD Operacoes para o Book de Trading
def captura_bd_operacoes(caminho_bd=BD_OPERACOES_DIR, col_inicio_vol=COL_INICIO_VOLUME, col_inicio_preco=COL_INICIO_PRECO,qtd_meses=MESES_BD_OPERACOES):

    bd_operacoes_arq = pd.read_excel(caminho_bd, sheet_name="BD_COMERCIAL", skiprows=3, usecols="B:FK")
    bd_operacoes_arq.dropna(subset=['COD_WBC'], inplace=True)

    # Monta um Dataframe com os meses e as horas dos meses
    df_periodos_e_horas = _periodo_e_horas_meses()

    # Transformando os COD WBC em unicos 
    bd_operacoes_arq['Duplicados'] = bd_operacoes_arq['COD_WBC'].duplicated(keep=False)
    cod_wbc_duplicados = 0
    todos_cod_wbc = []

    for i, linha in bd_operacoes_arq.iterrows():

        if linha['Duplicados']:
            todos_cod_wbc.append(f'wbc_{cod_wbc_duplicados}')
            cod_wbc_duplicados += 1

        else:
            todos_cod_wbc.append(linha['COD_WBC'])
        
    bd_operacoes_arq['COD_WBC'] = todos_cod_wbc

    # Monta o DF de Volumes da BD Operações,transpondo os volumes para linha
    bd_operacoes_vol = bd_operacoes_arq.iloc[:, col_inicio_vol : col_inicio_vol + qtd_meses]
    bd_operacoes_vol.columns = df_periodos_e_horas['Periodo'].to_list()
    bd_operacoes_vol['COD_WBC'] = bd_operacoes_arq['COD_WBC']
    bd_operacoes_vol = bd_operacoes_vol.melt(id_vars=["COD_WBC"],var_name="Periodo",value_name="Vol MWm")
    bd_operacoes_vol = bd_operacoes_vol.set_index(['COD_WBC','Periodo'])

    # Monta o DF de Precos da BD Operações, transpondo os precos para linha
    bd_operacoes_preco = bd_operacoes_arq.iloc[:, col_inicio_preco : col_inicio_preco + qtd_meses]
    bd_operacoes_preco.columns = df_periodos_e_horas['Periodo'].to_list()
    bd_operacoes_preco['COD_WBC'] = bd_operacoes_arq['COD_WBC']
    bd_operacoes_preco = bd_operacoes_preco.melt(id_vars=["COD_WBC"],var_name="Periodo",value_name="Preco")
    bd_operacoes_preco = bd_operacoes_preco.set_index(['COD_WBC','Periodo'])

    # Unifica os DFs de volume e preco
    bd_operacoes_vol_e_preco = bd_operacoes_vol.join(bd_operacoes_preco,on=['COD_WBC','Periodo']).reset_index().set_index('COD_WBC')

    # Monta o DF com as informações de cada contrato, sem volume e preco
    bd_operacoes_sem_vol_preco = bd_operacoes_arq.iloc[:,:col_inicio_vol]
    bd_operacoes_sem_vol_preco = bd_operacoes_sem_vol_preco.join(bd_operacoes_arq['Compra/venda']).join(bd_operacoes_arq['_'])
    bd_operacoes_sem_vol_preco = bd_operacoes_sem_vol_preco.set_index('COD_WBC')

    # Monta o DF final com todas as operações, as informacoes, volumes e precos
    bd_operacoes = bd_operacoes_sem_vol_preco.join(bd_operacoes_vol_e_preco, on='COD_WBC').reset_index(drop=True)
    bd_operacoes = bd_operacoes.dropna(subset=['Vol MWm', 'Preco']).set_index('COD_WBC').reset_index(drop=True)

    #Salva o Dataframe
    _salva_dataframes(df=bd_operacoes, nome="bd_operacoes",formato='Excel')

    # Retorna um Dataframe
    return bd_operacoes

ETL BD OPERACOES

In [ ]:
def etl_bd_operacoes(df, book= 'Trading'):
    

In [53]:
df1 = bd_operacoes_modificada

# Filtra para somente pegar um book
df1 = df1[df1['BOOK'] == 'Trading']

# Une os DFs da BD operacoes com o Periodo e horas e com o Precos do Book
df_uniao_bd_e_periodo = df1.merge(periodos_e_horas, left_on='Periodo', right_on='Periodo', how='left')
df_uniao_bd_e_precos_book = df1.merge(precos_book, left_on='Periodo', right_on='Periodo', how='left')

# Modificação do Preco
linhas_de_pld_mais = df_uniao_bd_e_precos_book['MOD PREÇO'] == 'PLD + Spread'
df1.loc[linhas_de_pld_mais,'MOD PREÇO'] = df_uniao_bd_e_precos_book.loc[linhas_de_pld_mais,'Preco'] + df_uniao_bd_e_precos_book.loc[linhas_de_pld_mais,'Valor']

# Preco com IPCA = Ajustado
linhas_de_preco_ajustado =  ~pd.isnull(df_uniao_bd_e_precos_book['ÍNDICE'])
df1.loc[linhas_de_preco_ajustado,'MOD PREÇO'] = 'Ajustado'


# Preenche com as horas, Volume em MWh e o PLD (Preço de mercado)
df1['Horas'] = df_uniao_bd_e_periodo['Horas']
df1['Vol MWh'] = df1['Vol MWm'] * df_uniao_bd_e_periodo['Horas']
df1['PLD'] = df_uniao_bd_e_precos_book['Valor']

#Cria colunas vazias
df1['Preco MTM'] = [pd.NaT] * len(df1)
df1['CNPJ Contraparte'] = [pd.NaT] * len(df1)

# PLD+
linhas_de_pld_mais = df_uniao_bd_e_precos_book['MOD PREÇO'] == 'PLD + Spread'
df1.loc[linhas_de_pld_mais,'MOD PREÇO'] = df_uniao_bd_e_precos_book.loc[linhas_de_pld_mais,'Preco'] + df_uniao_bd_e_precos_book.loc[linhas_de_pld_mais,'Valor']

# Venda
linhas_de_venda = df_uniao_bd_e_precos_book['Compra/venda'] == 'V'
df1.loc[linhas_de_venda,'Preco MTM'] = df_uniao_bd_e_precos_book.loc[linhas_de_venda,'Preco'] - df_uniao_bd_e_precos_book.loc[linhas_de_venda,'Valor']
df1.loc[linhas_de_venda,'CNPJ Contraparte'] = df_uniao_bd_e_precos_book.loc[linhas_de_venda,'CNPJ COMPRADOR'] 

# Compra
linhas_de_compra = df_uniao_bd_e_precos_book['Compra/venda'] == 'C'
df1.loc[linhas_de_compra,'Preco MTM'] = df_uniao_bd_e_precos_book.loc[linhas_de_compra,'Valor'] - df_uniao_bd_e_precos_book.loc[linhas_de_compra,'Preco']
df1.loc[linhas_de_venda,'CNPJ Contraparte'] = df_uniao_bd_e_precos_book.loc[linhas_de_venda,'CNPJ VENDEDOR'] 

df1['Receita MTM'] = df1['Vol MWh'] * df1['Preco MTM']

IndexingError: (0      False
1      False
2      False
3      False
4      False
       ...  
852    False
853    False
854     True
855     True
856     True
Name: MOD PREÇO, Length: 857, dtype: bool, 'MOD PREÇO')

In [105]:
df = bd_operacoes
filtro_1 = (df['SUBMERCADO'] == "SE")
filtro_2 = (df['FONTE'] == "Conv")
df.loc[filtro_1 & filtro_2, 'Vol. MWh'].sum()

5344519.508928

In [93]:
df = bd_operacoes
filtro_1 = (df['COD_WBC'] == 11691)
filtro_2 = (df['Periodo'].dt.strftime("%Y-%m") == "2023-01")
df.loc[filtro_1 & filtro_2, 'Vol. MWh'].sum()

13950.0

Forecast Assumpations

In [19]:
# Funcao para decidir qual é o ultimo dia util antes da data enviada
def _ultimo_dia_util(formato='data'):
    calendario_br = Brazil()

    try:
        dia = datetime.today().date() - pd.DateOffset(days=1)

        while calendario_br.is_working_day(dia) == False:
            dia = pd.to_datetime(dia) - pd.DateOffset(days=1)
        
        if formato == 'str':
            dia_util = dia.strftime('%Y-%m-%d')
        elif formato == 'data':
            dia_util = dia.date()
            
        print(f'> O ultimo dia util foi : {dia_util}')

    except:
        print("Formatação da data incorreta: 'AAAA-MM-DD'")

    # Retorna uma data
    return dia_util

In [55]:
# Monta a curva forward do EHUB
def ultima_curva_forward():

    #Define o ultimo dia util antes de hoje
    dia_util = _ultimo_dia_util('str')

    # Dataframe da Curva Forward do EHUB 
    curva_forward = pd.DataFrame(forwardCurve(dia_util, EHUB_EMAIL, EHUB_SENHA))

    #ETL da Curva Forward do EHUB
    periodo = []
    for linha in curva_forward['vertexDate']:
        data = date(year= pd.to_datetime(linha).year, month=pd.to_datetime(linha).month, day=1)
        periodo.append(data)

    curva_forward['Periodo'] = periodo
    curva_forward['Periodo'] = curva_forward['Periodo'].astype("datetime64")

    print('> Curva forward calculado com sucesso')

    #Salva o Dataframe
    _salva_dataframes(df=curva_forward, nome='curva_forward', formato='Excel')

    # Retorna um Dataframe
    return curva_forward

In [21]:
# Monta a curva de precos realizados da Forecast Assumptions
def precos_realizados_forecast_assumptions(mes_inicial= MES_INICIAL, forecast_assumption_diretorio= FORECAST_ASSUMPTIONS_DIR):
    
    # Dataframe do arquivo de Forecast Assumption com os PLDs realizados 
    forecast_assumptions_arq = pd.read_excel(forecast_assumption_diretorio, skiprows=1, sheet_name="Assumptions")

    # Checa se o primeiro mes do Book é igual ao primeiro mes do Forecast Assumptions
    if forecast_assumptions_arq.columns[3] != pd.to_datetime(mes_inicial):
        print('> Mes inicial da Forecast Assumptions diferente do Mes inicial do Book de Trading')
        print('> Continuando sem realizado do PLD')
        
        forecast_assumptions = None
    else:
    # ETL do Forecast Assumptions
        forecast_assumptions = forecast_assumptions_arq.iloc[:,1:15].loc[7:,:].reset_index(drop=True)
        forecast_assumptions['FORECAST ASSUMPTIONS'] = forecast_assumptions.iloc[:,1]
        forecast_assumptions['FORECAST ASSUMPTIONS'].replace({'Previous Forecast': pd.NaT , 'Forecast': pd.NaT }, inplace=True)
        forecast_assumptions['FORECAST ASSUMPTIONS'].ffill(inplace=True)
        forecast_assumptions.rename(columns={'Unnamed: 2': 'Tipo'}, inplace=True)
        forecast_assumptions = forecast_assumptions[~forecast_assumptions.iloc[:,3].isna()]
        forecast_assumptions = pd.melt(forecast_assumptions, id_vars=['FORECAST ASSUMPTIONS', 'Tipo'],var_name='Periodo', value_name="Valor" )
        print('> Precos realizados calculado com sucesso')

    #Salva o Dataframe
    _salva_dataframes(df=forecast_assumptions, nome='forecast_assumptions', formato='Excel')

    # Retorna um Dataframe
    return forecast_assumptions

In [22]:
# Monta a curva de precos que vai ser utilizada no Book de Trading
def curva_precos_para_book(mes_inicial= MES_INICIAL):

    # Roda a funcao da ultima curva forward
    curva_forward =  ultima_curva_forward()

    # Roda a funcao da dos precos realizados
    forecast_assumptions =  precos_realizados_forecast_assumptions()

    # Executa a funcao do periodo do book
    periodo_book = _periodo_e_horas_meses()

    # Define o mes inicial do Book
    mes_inicial = pd.to_datetime(mes_inicial)

    # Se o forecast_assumptions for None quer dizer que não tem PLD realizado ainda e 
    # Se eu não tiver o mes inicial na Curva Foward
    if forecast_assumptions.empty and (mes_inicial in curva_forward['Periodo']) == False:
        
        print('> Não consigo montar os precos do Book pois não tenho os PLDS realizados nem a curva forward')
        print('> Favor criar uma planilha igual ao Forecast Assumptions com a data correta e o PLD realizado')
    
    # Se somente o forecast_assumptions for None
    else:
        
        # ETL da curva forward para o preco do Book
        df_curva_forward = curva_forward[['dataSource', 'name', 'Periodo', 'vertexValue']]
        df_curva_forward = df_curva_forward.rename(columns={'dataSource': 'Fonte', 'name': 'Tipo', 'vertexValue': 'Valor'})
        df_curva_forward['Fonte'] = 'Ehub BBCCE'
        df_curva_forward['Tipo'] = 'Forward'
        df_curva_forward = df_curva_forward[df_curva_forward['Periodo'].isin(periodo_book['Periodo'])]

        # Aumenta o periodo até coincidir com o periodo do Book
        while not df_curva_forward['Periodo'].iloc[-1] == periodo_book['Periodo'].iloc[-1]:

            ult_linha = df_curva_forward.iloc[-1].copy()
            ult_linha['Periodo'] = pd.to_datetime(ult_linha['Periodo']) + pd.DateOffset(months=1)
            df_curva_forward = df_curva_forward.append([ult_linha])
        
        precos_book = df_curva_forward.sort_values(by='Periodo').reset_index(drop=True)
        
        # Se somente o forecast_assumptions for None
        if not forecast_assumptions.empty:

            # ETL dos precos realizados para o preco do Book
            filtro_1 = (forecast_assumptions['FORECAST ASSUMPTIONS'] == 'PLD SE')
            filtro_2 = (forecast_assumptions['Tipo'] == "Forecast")
            df_forecast_assumptions = forecast_assumptions.loc[filtro_1 & filtro_2].rename(columns={'FORECAST ASSUMPTIONS': 'Fonte' })
            df_forecast_assumptions = df_forecast_assumptions[~df_forecast_assumptions['Periodo'].isin(df_curva_forward['Periodo'])]
            df_forecast_assumptions['Fonte'] = 'Forecast Assumptions'
            df_forecast_assumptions['Tipo'] = 'Realizado'

            precos_book = pd.concat([df_curva_forward, df_forecast_assumptions]).sort_values(by='Periodo').reset_index(drop=True)
        
        print('> Precos para o Book montado com sucesso')

    #Salva o Dataframe
    _salva_dataframes(df=precos_book, nome='precos_book', formato='Excel')

    # Retorna um Dataframe
    return precos_book

Reajuste Inflacao

In [51]:
infla_ipca = pd.read_pickle(f'{INFLACOES_DFS_DIR}/Inflacao_IPCA.df')
infla_igpm = pd.read_pickle(f'{INFLACOES_DFS_DIR}/Inflacao_IGPM.df')
bd_operacoes_modificada = pd.read_excel(f'{SALVA_OS_DFS}/bd_operacoes.xlsx')
precos_book = pd.read_excel(f'{SALVA_OS_DFS}/precos_book.xlsx')
periodos_e_horas = pd.read_excel(f'{SALVA_OS_DFS}/periodos_e_horas.xlsx') 

In [47]:
bd_operacoes_modificada['DATA BASE'].fillna(0).unique()

array([Timestamp('2020-06-01 00:00:00'), Timestamp('2020-07-01 00:00:00'),
       Timestamp('2020-09-16 00:00:00'), Timestamp('2020-10-01 00:00:00'),
       Timestamp('2020-12-01 00:00:00'), Timestamp('2020-12-21 00:00:00'),
       Timestamp('2021-04-12 00:00:00'), Timestamp('2021-04-13 00:00:00'),
       Timestamp('2021-04-14 00:00:00'), Timestamp('2021-05-14 00:00:00'),
       Timestamp('2021-05-01 00:00:00'), Timestamp('2021-06-18 00:00:00'),
       Timestamp('2021-06-01 00:00:00'), Timestamp('2021-07-01 00:00:00'),
       Timestamp('2021-07-09 00:00:00'), Timestamp('2021-07-13 00:00:00'),
       Timestamp('2021-08-01 00:00:00'), Timestamp('2021-07-29 00:00:00'),
       Timestamp('2021-08-09 00:00:00'), Timestamp('2021-09-02 00:00:00'),
       0, Timestamp('2021-09-21 00:00:00'),
       Timestamp('2021-11-01 00:00:00'), Timestamp('2021-12-21 00:00:00'),
       Timestamp('2021-12-23 00:00:00'), Timestamp('2021-12-28 00:00:00'),
       Timestamp('2022-03-18 00:00:00'), Timestamp('2022

In [43]:
bd_operacoes_modificada.iloc[:5,12:]

,INICIO,FIM,VOL (MWm),SPREAD,MOD PREÇO,PAGAMENTO,GARANTIAS,ÍNDICE,DATA BASE,VIGENTE,ESTIMADO,BBCE,SSJ,Enviada,Arquivada,Compra/venda,_,Periodo,Vol MWm,Preco
0,2023-01-01,2023-12-31,18.75,150.0,Fixo,6º du,Garantia Corporativa,IPCA,2020-06-01,NaN,NaN,NaN,SSJ 8489,Digital,2021-11-17 00:00:00,C,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,2023-01-01,18.75,150.0
1,2023-01-01,2023-12-31,18.75,150.0,Fixo,6º du,Garantia Corporativa,IPCA,2020-06-01,NaN,NaN,NaN,SSJ 8489,Digital,2021-11-17 00:00:00,C,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,2023-02-01,18.75,150.0
2,2023-01-01,2023-12-31,18.75,150.0,Fixo,6º du,Garantia Corporativa,IPCA,2020-06-01,NaN,NaN,NaN,SSJ 8489,Digital,2021-11-17 00:00:00,C,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,2023-03-01,18.75,150.0
3,2023-01-01,2023-12-31,18.75,150.0,Fixo,6º du,Garantia Corporativa,IPCA,2020-06-01,NaN,NaN,NaN,SSJ 8489,Digital,2021-11-17 00:00:00,C,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,2023-04-01,18.75,150.0
4,2023-01-01,2023-12-31,18.75,150.0,Fixo,6º du,Garantia Corporativa,IPCA,2020-06-01,NaN,NaN,NaN,SSJ 8489,Digital,2021-11-17 00:00:00,C,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,2023-05-01,18.75,150.0
